Четвёртый принцип — принцип разделения интерфейсов. Это в каком-то смысле перенос SRP на интерфейсы. Мы говорим, что любой используемый интерфейс должен быть как можно меньше; не включать в себя лишние методы.

Например, пусть у нас есть функция, которая принимает на вход список и удаляет оттуда falsy элементы:

In [1]:
def remove_falsy(l):
    return [x for x in l if x]

In [2]:
print(remove_falsy([1, 2, 3, 0, 4]))

[1, 2, 3, 4]


Мы можем сформулировать требование `remove_falsy` так: на вход подаётся список. В частности, у списка должно быть можно получить длину, элемент по номеру, пройтись по всем элементам...

Это довольно серьёзное требование. Например, генераторы и `range` ему не удовлетворяют и это нам не мешает:

In [3]:
print(remove_falsy(range(-10, 10)))

[-10, -9, -8, -7, -6, -5, -4, -3, -2, -1, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [4]:
print(remove_falsy(x % 5 for x in range(20)))

[1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4]


То есть здесь мы потребовали больше, чем нам реально надо. В Python это в каком-то смысле нестрашно: у нас нет статической проверки типов. Мы можем нагло скормить `remove_falsy` что-то, не соответствующее контракту, и всё отработает. В языках вроде Java/C++ это будет критично: если `remove_falsy` может принимать только `vector`, то мы не сможем туда скормить `list`, массив и прочие последовательности элементов.

Правильное решение — ввести интерфейс "что-то, по чему можно итерироваться" (так как `remove_falsy` нужно ровно это) и потребовать в контракте его.

В случае C++, например, стандартное решение в STL — требовать на вход не вектора, а пары итераторов. Тогда алгоритмы вроде `std::lower_bound` одинаково работают и на векторах, а на массивах, и даже на списках (правда, уже за линейное время).

Чуть более бытовой (или абстрактный?) пример: пусть мы пишем игру и у есть появился интерфейс "игровой объект", у которого есть:

1. Положение на карте.
1. Количество здоровья.
1. Свойство "сколько места занимает в инвентаре" (для предметов, которые можно поднять).
1. Метод "сходить" (для существ, которые могут передвигаться самостоятельно).
1. Стоимость (для предметов, которые можно продать).
1. Количество элементов внутри (для сундуков с предметами и врагов с лутом).

Наверняка даже все объекты можно этому интерфейсу подчинить.
Например, условный "враг" может занимать $+\infty$ места в инвентаре (никогда не положить), стоимость может быть равна $-\infty$ (не продать), количество элементов внутри может быть тоже равну нулю (или количеству лута).
А для аптечки бессмысленный метод "сходить" может просто ничего не делать.

Таким образом, у нас даже получится более-менее адекватно реализовать все методы.
Однако это нарушает ISP: у нас для половины объектов метод "сходить" на самом деле смысла не имеет.
И по-хорошему надо этот большой интерфейс "игровой объект" разделить на несколько.
Например:

1. Игровой объект: положение на карте, количество здоровья.
1. Поднимаемый игровой объект: игровой объект + "сколько места занимает".
1. Продаваемый игровой объект: поднимаемый игровой объект + "сколько стоит".
1. Живой игровой объект: игровой объект + метод "сходить".
1. Игровой объект с внутренностями: игровой объект + "сколько элементов внутри".

Тут у нас интерфейсы получаются более перпендикулярны и каждый объект реализует только нужные интерфейсы.
Не надо придумывать магические константы для $\infty$ и не надо как-то специально их показывать на карте.